<a href="https://colab.research.google.com/github/namoshi/dl_intro/blob/master/MLP_mnist_visualize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualize the features and parameters of the trained MLP



In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import numpy as np

In [2]:
# load MNIST data
train_Dataset = dsets.MNIST(
      root='./data',  # rootで指定したフォルダーを作成して生データを展開
      train=True,  # 学習かテストかの選択
      transform=transforms.ToTensor(),  # PyTroch のテンソルに変換
      download=True)  # ダウンロードするかどうかの選択

test_dataset = dsets.MNIST(
      root='./data',
      train=False,
      transform=transforms.ToTensor(),
      download=True)

train_dataset, valid_dataset = torch.utils.data.random_split(  # データセットの分割
      train_Dataset,   # 分割するデータセット
      [48000, 12000])  # 分割数

print('train_dataset = ', len(train_dataset))
print('valid_dataset = ', len(valid_dataset))
print('test_dataset = ', len(test_dataset))


train_dataset =  48000
valid_dataset =  12000
test_dataset =  10000


In [13]:
# set data loader
batch_size=64
train_loader = torch.utils.data.DataLoader(
      dataset=train_dataset,  # データセットの指定
      batch_size=batch_size,  # ミニバッチの指定
      shuffle=True,  # シャッフルするかどうかの指定
      num_workers=2)  # コアの数

valid_loader = torch.utils.data.DataLoader(
      dataset=valid_dataset,
      batch_size=batch_size,
      shuffle=False,
      num_workers=2)

test_loader = torch.utils.data.DataLoader(
      dataset=test_dataset,
      batch_size=batch_size,
      shuffle=False,
      num_workers=2)

In [4]:
# Multi Layer Perceptron Network
class MLPNet (nn.Module):
    def __init__(self):
        super(MLPNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28 * 1, 128)
        self.fc2 = nn.Linear(128, 10)
#        self.dropout = nn.Dropout2d(0.4)

    def forward(self, x):
        y = torch.relu(self.fc1(x))
#        y = torch.sigmoid(self.fc1(x))
#        y = torch.tanh(self.fc1(x))
#        y = self.dropout1(y)
        z = self.fc2(y)
        return z


In [10]:
# Load the saved model from the file

# select device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = MLPNet().to(device)
if device == 'cpu':
    net = torch.load('./trained_models/MLP_mnist.pth', map_location=torch.device('cpu'), weights_only=False)
else:
    net = torch.load('./trained_models/MLP_mnist.pth', weights_only=False)
print(device)
print(net)

# optimizing
criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=0.01)
optimizer = optim.AdamW(net.parameters(), lr=0.01, weight_decay=0.01)


cpu
MLPNet(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [19]:
# ======== eval_mode (train samples) ======
net.eval()
features = np.zeros((len(train_dataset), 10))

with torch.no_grad():
    total = 0
    acc = 0
    i = 0
    for images, labels in train_loader:        
        images, labels = images.view(-1, 28 * 28 * 1 ).to(device), labels.to(device)
        outputs = net(images)
        acc += (outputs.max(1)[1] == labels).sum().item()
        total += labels.size(0)
        len_batch = len(images)
        features[i:i+len_batch] = outputs.cpu().detach().numpy()
        i += len_batch
    print('train_accuracy: {} %'.format(100 * acc / total)) 

print(features)

train_accuracy: 98.425 %
[[ -9.31153774  -4.2671051    8.68529224 ...  -0.1514914   -4.18324566
  -13.93063068]
 [-17.93710899  -7.66156292 -14.93036938 ... -24.71265793  -5.41768265
  -10.99696064]
 [ -2.97955799  -9.89301872  -8.38357735 ... -15.94122791  -0.09913111
  -17.8880024 ]
 ...
 [-10.2529068   -0.79119664 -11.30435562 ... -24.25601196  16.50928688
   -7.14565563]
 [-11.45661163  -6.23023129 -15.30533218 ... -25.91217804  21.79232407
   -9.28656483]
 [-39.35433578 -28.89614868 -20.05836487 ...  -7.76903009 -11.65831852
   13.68480873]]


# PCA for output features